In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import inspect
from PIL import Image

In [ ]:
import glob
n_people = 6
n_data = 0
image_size = 32

t_train = np.array([])
for n in range(n_people): #人数分の顔画像についてのループ
    if n==3: continue
    files = glob.glob('drive/My Drive/Colab Notebooks/創造性/data/Identification_persons/' + str(n) + '/*')
    for i in range(len(files)):
        img = Image.open(files[i])
        img = img.resize((image_size, image_size)) #image_size×image_size 画素に変換
        data = np.array(img) # numpy 形式に変換する
        data = data.reshape([-1, image_size, image_size])
        #print(data)

        if i==0 and n==0:
            x_train = data
        else :
            x_train = np.append(x_train, data, axis=0) #行ベクトルを連結する

        t_train = np.hstack((t_train, np.array([n]))) #行ベクトルを連結する
    print(t_train.shape[0] - n_data)
    n_data = t_train.shape[0]

#X = np.asarray(X_list).reshape(-1,imY*imX)
#各配列の大きさを確認する
print(x_train.shape)
print(t_train.shape)

In [ ]:
class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 1, image_size, image_size).astype('float32') / 255
        self.t_train = t_train
        print(self.x_train.shape)

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(self.t_train[idx], dtype=torch.long)


trainval_data = train_dataset(x_train, t_train)

In [ ]:
batch_size = 16

val_size = 500
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
import numpy as np
import torch

rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Conv(nn.Module):
    def __init__(self, filter_shape, function=lambda x: x, stride=(1, 1), padding=0):
        super().__init__()
        # Heの初期値
        fan_in = filter_shape[1] * filter_shape[2] * filter_shape[3]
        # filter_shape: (出力チャンネル数)x(入力チャンネル数)x(縦の次元数)x(横の次元数)
        fan_out = filter_shape[0] * filter_shape[2] * filter_shape[3]

        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        -np.sqrt(6/fan_in),
                        np.sqrt(6/fan_in),
                        size=filter_shape
                    ).astype('float32')))

        # バイアスはフィルタごとなので, 出力フィルタ数と同じ次元数
        self.b = nn.Parameter(torch.tensor(np.zeros((filter_shape[0]), dtype='float32')))
        self.function = function
        self.stride = stride
        self.padding = padding
        
    def forward(self, x):
        u = F.conv2d(x, self.W, bias=self.b, stride=self.stride, padding=self.padding)
        return self.function(u)



class Pooling(nn.Module):
    def __init__(self, ksize=(2, 2), stride=(2, 2), padding=0):
        super().__init__()
        self.ksize = ksize
        self.stride = stride
        self.padding = padding

    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=self.ksize, stride=self.stride, padding=self.padding)


class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.size()[0], -1)


class Dense(nn.Module):
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        super().__init__()
        # He Initialization
        # in_dim: 入力の次元数、out_dim: 出力の次元数              
        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        -np.sqrt(6/in_dim),
                        np.sqrt(6/in_dim),
                        size=(in_dim, out_dim)
                    ).astype('float32')))

        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]).astype('float32')))
        self.function = function

    def forward(self, x):
        return self.function(torch.matmul(x, self.W) + self.b)

class Activation(nn.Module):
    def __init__(self, function=lambda x: x):
        super().__init__()
        self.function = function

    def __call__(self, x):
        return self.function(x)

class BatchNorm(nn.Module):
    def __init__(self, shape, epsilon=np.float32(1e-5)):
        super().__init__()
        self.gamma = nn.Parameter(torch.tensor(np.ones(shape, dtype='float32')))
        self.beta = nn.Parameter(torch.tensor(np.zeros(shape, dtype='float32')))
        self.epsilon = epsilon

    def forward(self, x):
        mean = torch.mean(x, (0, 2, 3), keepdim=True)
        std = torch.std(x, (0, 2, 3), keepdim=True)
        x_normalized = (x - mean) / (std**2 + self.epsilon)**0.5
        return self.gamma * x_normalized + self.beta

class Dropout(nn.Module):
    def __init__(self, dropout_ratio=0.5):
        super().__init__()
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x):
        # 学習時はdropout_ratio分だけ出力をシャットアウト
        if self.training:
            self.mask = torch.rand(*x.size()) > self.dropout_ratio
            return x * self.mask.to(x.device)
        # 推論時は出力に`1.0 - self.dropout_ratio`を乗算することで学習時の出力の大きさに合わせる
        else:
            return x * (1.0 - self.dropout_ratio)


'''
conv_net = nn.Sequential(
    Conv((32, 1, 31, 31)),     # 画像の大きさ：100x100x1 -> 70x70x32
    BatchNorm((32, 70, 70)),
    Activation(F.relu),
    Dropout(),
    Pooling((2, 2),                  # 70x70x32 -> 35x35x32
    Conv((64, 32, 21, 21)),     # 35x35x32 -> 15x15x64
    BatchNorm((64, 15, 15)),
    Activation(F.relu),
    Dropout(),
    Pooling((2, 2)),                 # 15x15x64 -> 7x7x64
    Conv((128, 64, 4, 4)),           # 7x7x64 -> 4x4x128
    BatchNorm((128, 4, 4)),
    Activation(F.relu),
    Pooling((2, 2)),                 # 4x4x128 -> 2x2x128
    Flatten(),
    Dense(2*2*128, 256, F.relu),
    Dense(256, 10)
)

conv_net = nn.Sequential(
    Conv((32, 1, 8, 8)),     # 画像の大きさ：40x40x1 -> 33x33x32
    Dropout(),
    Pooling((3, 3),(3, 3)),                  # 33x33x32 -> 11x11x32
    Conv((64, 32, 6, 6)),          # 11x11x32 -> 6x6x64
    Conv((128, 64, 3, 3)),           # 6x6x64 -> 4x4x128
    Activation(F.relu),
    Pooling((2, 2)),                 # 4x4x128 -> 2x2x128
    Flatten(),
    Dense(2*2*128, 256, F.relu),
    Dense(256, 10)
)
'''
conv_net = nn.Sequential(
    Conv((20, 1, 5, 5)),     # 画像の大きさ：32x32x1 -> 28x28x20
    Pooling((2, 2)),                  # 28x28x20 -> 14x14x20
    Conv((20, 20, 5, 5)),          # 14x14x20 -> 10x10x20
    Pooling((2, 2)),                  # 10x10x20 -> 5x5x20
    Conv((20, 20, 3, 3)),           # 5x5x20 -> 3x3x20
    Flatten(),
    Dense(3*3*20, 20, F.relu),
    Dense(20, 6)
)



def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)


conv_net.apply(init_weights)


n_epochs = 10
lr = 0.001
device = 'cuda'

conv_net.to(device)
optimizer = optim.Adam(conv_net.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    conv_net.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]

        conv_net.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = conv_net.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    conv_net.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = conv_net.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

In [ ]:
#torch.save(conv_net.state_dict(), 'drive/My Drive/Colab Notebooks/創造性/Identification_persons.pth')

In [ ]:
conv_net.state_dict()